# 일반 구매 차감 
1000만 세톱 데이터에 대해 일반 구매 차감 연산 실행한 결과  
 - 전체 : 94 초  
 - 세션 생성 : 23 초  
 - 차감 계산 : 37 초 (14초.캐시)    
 - 스냅샷 저장 : 32 초  
 - 결과 확인 : 2 초  
 - 요약 결과 생성 : 
 - 요약 rdb 기록 : 


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = f'spark-02-load-inven-6m'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = f'inven/table-set-6m-50-1000'
file_format = 'parquet'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '4g')\
    .config('spark.num.executors', '4')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '2g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [4]:
!ls -l /hive-bin/lib | grep mysql

-rw-r--r-- 1 root root   1006906 Mar 10 06:57 mysql-connector-java-5.1.49-bin.jar
-rw-r--r-- 1 root staff    10476 Nov 15  2018 mysql-metadata-storage-0.12.0.jar


In [5]:
%%time
spark = spark_creation()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


CPU times: user 50.4 ms, sys: 8.56 ms, total: 59 ms
Wall time: 22.9 s


## 전체 구매 차감  
모든 세탑에 (지정 비율 * 구매 단위) 값을 차감  

In [6]:
%%time 
# sparkSQL로 전체 데이터 읽어서 카운트  
# 건 수 차이 없는 방식. 최초, 재실행 여부 차이.  
# 최초 : 22 초 , 재실행 : 13 초   
## 구매 요청 값 
req_val = -10 
## 인벤 기준 정보 조회  
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
## 요청 구매 단위 * 비율 계산  
sql_req_rate = f"select setop, inv_rate_01*{req_val} inv_req_01, inv_rate_02*{req_val} inv_req_02, inv_rate_03*{req_val} inv_req_03\
, inv_rate_04*{req_val} inv_req_04, inv_rate_05*{req_val} inv_req_05, inv_rate_06*{req_val} inv_req_06 from setop_view"
spark.sql(sql_req_rate).createOrReplaceTempView('setop_req_rate')

# 차감 요청 정보를 원본에 반영  
sql_calc = """
select setop
, sum(inv_req_01) inv_req_01 , sum(inv_req_02) inv_req_02
, sum(inv_req_03) inv_req_03 , sum(inv_req_04) inv_req_04
, sum(inv_req_05) inv_req_05 , sum(inv_req_06) inv_req_06
from 
(select setop
, inv_req_01, inv_req_02, inv_req_03
, inv_req_04, inv_req_05, inv_req_06
from setop_view 
UNION ALL 
select setop
, inv_req_01, inv_req_02, inv_req_03
, inv_req_04, inv_req_05, inv_req_06
from setop_req_rate
) as A 
group by setop 
"""
spark.sql(sql_calc).createOrReplaceTempView('result')

# 계산 결과와 원본 비교  
spark.sql("select count(1) cnt, sum(inv_req_01+ inv_req_02+ inv_req_03) from setop_view").show()
spark.sql("select count(1) cnt, sum(inv_req_01+ inv_req_02+ inv_req_03) from result").show()

+--------+---------------------------------------------+
|     cnt|sum(((inv_req_01 + inv_req_02) + inv_req_03))|
+--------+---------------------------------------------+
|10000000|                                            0|
+--------+---------------------------------------------+

+--------+---------------------------------------------+
|     cnt|sum(((inv_req_01 + inv_req_02) + inv_req_03))|
+--------+---------------------------------------------+
|10000000|                                       -3.0E8|
+--------+---------------------------------------------+

CPU times: user 16 ms, sys: 5.93 ms, total: 21.9 ms
Wall time: 35.8 s


In [6]:
%%time
# 차감한 모든 결과를 result table로 스냅샷 기록  
result = spark.sql("select * from result")
result.write.save(path="result", format=file_format, mode='overwrite')

CPU times: user 11.4 ms, sys: 629 µs, total: 12.1 ms
Wall time: 32.9 s


In [7]:
%%time
cnt = spark.read.format(file_format).load('result').count()
print(f'Row count = {cnt:,}')

Row count = 10,000,000
CPU times: user 3.71 ms, sys: 0 ns, total: 3.71 ms
Wall time: 1.7 s


In [ ]:
# rdb 기록할 요약 결과 생성 : 채널/시간/주중, 지역별, 주제별   
# 우선 hdfs 에 먼저 기록 ; 비율, inv값, 청약량  
# 지역별/시간별/주말 ...... 
# 주제별 요약 : 여러 주제에 할당 되기 때문에, 실제 시간의 몇 배가 나올 듯...  차감하지 않은 주제가 차감되는 현상도 발생한다.   
# 비율, inv 값은 원본 테이블의 값. 청약량만 계산 결과.  
# -> 일반 구매시 : 채널, 지역, 주제 테이블 차감.  
# -> 주제 구매시 : ...
# -> 2 경우 모두, setop table에서 차감하고 결과를 요약으로 생성하는 방법만 다른 듯... 
## 1. 주제 구매시 채널/시간/주중 차감은 비율 적용해서 ... 
##   - 지역 차감은 ? 지역은 세톱에 영향 받는 데... 주제 세탑 조회해서 여기에 다시 지역 비율 반영해야 할 듯... 
##   - 주제 차감은 세톱에서? 
## 2. 시간 구매시 지역/주제 차감은... 

In [14]:
# %%time
# 재실행해도 계산 누적되서 빨라지지 않는다.  
# spark.sql("select * from result").createOrReplaceTempView('setop_view')
# # 재실행 1회 : 14 초 .  5회 : 
# spark.sql(sql_calc).createOrReplaceTempView('result')

# # 계산 결과와 원본 비교  
# spark.sql("select count(1) cnt, sum(inv_req_01+inv_req_02+inv_req_03+inv_req_04+inv_req_05+inv_req_06) from setop_view").show()
# spark.sql("select count(1) cnt, sum(inv_req_01+inv_req_02+inv_req_03+inv_req_04+inv_req_05+inv_req_06) from result").show()

+--------+------------------------------------------------------------------------------------------+
|     cnt|sum((((((inv_req_01 + inv_req_02) + inv_req_03) + inv_req_04) + inv_req_05) + inv_req_06))|
+--------+------------------------------------------------------------------------------------------+
|10000000|                                                                                    -1.2E9|
+--------+------------------------------------------------------------------------------------------+

+--------+------------------------------------------------------------------------------------------+
|     cnt|sum((((((inv_req_01 + inv_req_02) + inv_req_03) + inv_req_04) + inv_req_05) + inv_req_06))|
+--------+------------------------------------------------------------------------------------------+
|10000000|                                                                                    -1.8E9|
+--------+-----------------------------------------------------------------------

In [32]:
# # mysql test_jdbc db 생성 
# set global validate_password_policy=LOW;
# set global validate_password_length=3;
# CREATE USER 'jdbc'@'%' IDENTIFIED BY 'jdbc';
# CREATE DATABASE test_jdbc;
# GRANT ALL privileges on test_jdbc.* to 'jdbc'@'%' with GRANT option;
# flush privileges;

#### 
# conf = pyspark.SparkConf().setAll([('spark.executor.id', 'driver'), 
#                                # ('spark.app.id', 'local-1631738601802'), 
#                                # ('spark.app.name', 'PySparkShell'), 
#                                # ('spark.driver.port', '32877'), 
#                                # ('spark.sql.warehouse.dir', 'file:/home/data_analysis_tool/spark-warehouse'), 
#                                # ('spark.driver.host', 'localhost'), 
#                                # ('spark.sql.catalogImplementation', 'hive'), 
#                                # ('spark.rdd.compress', 'True'), 
#                                # ('spark.driver.bindAddress', 'localhost'), 
#                                # ('spark.serializer.objectStreamReset', '100'), 
#                                # ('spark.master', 'local[*]'), 
#                                # ('spark.submit.pyFiles', ''), 
#                                # ('spark.app.startTime', '1631738600836'), 
#                                # ('spark.submit.deployMode', 'client'), 
#                                # ('spark.ui.showConsoleProgress', 'true'),
#                                # ('spark.driver.extraClassPath','/tmp/postgresql-42.2.23.jar')])

In [7]:
%%time
# rdb 쓰기 테스트 
# conf = SparkConf()  # create the configuration
#conf.set("spark.jars", "/path/to/postgresql-connector-java-someversion-bin.jar")  # set the spark.jars
# --driver-class-path postgresql-9.4.1207.jar --jars postgresql-9.4.1207.jar
# /hadoop/share/hadoop/yarn/lib 에 mysql jdbc.jar 넣는다.  
props = {"driver":"com.mysql.jdbc.Driver"}
db_url = "jdbc:mysql://rdb/test_jdbc?user=jdbc&password=jdbc"
tbl = "from_spark"
rdb = spark.sql('select * from result limit 10')
rdb.write.jdbc(db_url, tbl, mode='overwrite', properties=props)

In [ ]:
# jdbcDF = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql:dbserver") \
#     .option("dbtable", "jdbc:postgresql:dbserver") \
#     .load()

In [26]:
spark.stop()